In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

/Users/henriquerodrigues/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
import time
from selenium import webdriver
from urllib.parse import urljoin

In [5]:
from datetime import datetime, timedelta

In [3]:
driver = webdriver.Safari()
driver.get("https://www.bing.com/news/search?q=elon+musk&form=YFNR")
time.sleep(2)
scroll_pause_time = 1
screen_height = driver.execute_script("return window.screen.height;")
i = 1

while True:
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    if (screen_height) * i > scroll_height*10: #how much should i scroll
        break 

l=[]
o={}

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [13]:
completeData = soup.find_all("div",{"class":"news-card newsitem cardcommon"})

for i in range(0, len(completeData)):
    o["ARTICLE NUMBER"]=i+1
    o["TITLE"]=completeData[i].find("a",{"class":"title"}).text
    o["AUTHOR"]=completeData[i].find("a",{"class":"title"}).get("data-author")  
    o["DATE"]=completeData[i].find("span", {"aria-label": True}).get("aria-label") 
    o["DESCRIPTION"]=completeData[i].find("div",{"class":"snippet"}).text #try replacing with text
    o["URL"]=completeData[i].find("a", {"class":"title"}).get("href")
    l.append(o)
    o={}

In [14]:
table = pd.DataFrame(l)
table

,ARTICLE NUMBER,TITLE,AUTHOR,DATE,DESCRIPTION,URL
0,1,Elon Musk started a price war that Tesla can't...,Yahoo News UK,2 hours ago,Tesla has spent the past year slashing its pri...,https://uk.news.yahoo.com/elon-musk-started-pr...
1,2,Elon Musk 'bans safety clothes because he hate...,Daily Mail,1 day ago,Elon Musk discouraged employees from wearing y...,https://www.dailymail.co.uk/news/article-12737...
2,3,Obesity drug Wegovy taken by former UK prime m...,Daily Mail,22 hours ago,New research indicates the obesity drug Wegovy...,https://www.dailymail.co.uk/news/article-12738...
3,4,Members of the 'PayPal Mafia' include tech tit...,Yahoo,12 hours ago,"""PayPal mafia"" members Elon Musk, Peter Thiel,...",https://www.yahoo.com/lifestyle/members-paypal...
4,5,Elliot Page Gets Fan Vote To Play Elon Musk In...,Yahoo,20 hours ago,Tesla founder Elon Musk has a biopic on his li...,https://www.yahoo.com/entertainment/elliot-pag...
...,...,...,...,...,...,...
182,183,"Twitter, now X, to remove blocking feature - Musk",BBC,18/08/2023,The blocking feature will be removed for users...,https://www.bbc.co.uk/news/business-66550959?a...
183,184,Zuckerberg 'not holding breath' over Musk cage...,BBC,07/08/2023,Following a post on X in which Mr Musk claimed...,https://www.bbc.co.uk/news/business-66424364
184,185,Musk’s one-year X reign: how shocking has it b...,City A.M.,27/10/2023,"A year after Elon Musk’s Twitter takeover, now...",https://www.cityam.com/musks-one-year-x-reign-...
185,186,Regulator sues Musk to force testimony in X probe,BBC,05/10/2023,Financial regulators in the US are suing Elon ...,https://www.bbc.com/news/business-67024538


In [6]:
table.to_csv('output.csv', index=False)

In [15]:
extracted_texts = []
for url in table["URL"]:
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup_article = BeautifulSoup(response.text, 'html.parser')

        paragraphs = soup_article.find_all('p')
        text_content = ' '.join([paragraph.get_text() for paragraph in paragraphs])

        extracted_texts.append(text_content)
        
    except Exception as e:
        print(f"Error processing URL {url}: {str(e)}")
        extracted_texts.append(None)

table['TEXT'] = extracted_texts


Error processing URL https://www.ibtimes.com/elon-musk-distances-cryptocurrency-says-his-companies-will-never-create-crypto-token-3717580: 403 Client Error: Forbidden for url: https://www.ibtimes.com/elon-musk-distances-cryptocurrency-says-his-companies-will-never-create-crypto-token-3717580
Error processing URL https://www.ibtimes.co.uk/elon-musk-plans-integrate-ai-startup-xai-his-social-media-platform-x-1721339: 403 Client Error: Forbidden for url: https://www.ibtimes.co.uk/elon-musk-plans-integrate-ai-startup-xai-his-social-media-platform-x-1721339
Error processing URL https://www.screendaily.com/news/darren-aronofsky-to-direct-elon-musk-biopic-for-a24/5187791.article: 403 Client Error: Forbidden for url: https://www.screendaily.com/news/darren-aronofsky-to-direct-elon-musk-biopic-for-a24/5187791.article
Error processing URL https://www.ibtimes.co.uk/elon-musk-charge-new-users-fee-basic-account-x-1720737: 403 Client Error: Forbidden for url: https://www.ibtimes.co.uk/elon-musk-charg

In [16]:
table

,ARTICLE NUMBER,TITLE,AUTHOR,DATE,DESCRIPTION,URL,TEXT
0,1,Elon Musk started a price war that Tesla can't...,Yahoo News UK,2 hours ago,Tesla has spent the past year slashing its pri...,https://uk.news.yahoo.com/elon-musk-started-pr...,Elon Musk has started an electric-vehicle pric...
1,2,Elon Musk 'bans safety clothes because he hate...,Daily Mail,1 day ago,Elon Musk discouraged employees from wearing y...,https://www.dailymail.co.uk/news/article-12737...,By James Gordon For Dailymail.com and Reuters...
2,3,Obesity drug Wegovy taken by former UK prime m...,Daily Mail,22 hours ago,New research indicates the obesity drug Wegovy...,https://www.dailymail.co.uk/news/article-12738...,By Noa Halff For Dailymail.Com and Associated ...
3,4,Members of the 'PayPal Mafia' include tech tit...,Yahoo,12 hours ago,"""PayPal mafia"" members Elon Musk, Peter Thiel,...",https://www.yahoo.com/lifestyle/members-paypal...,A group of early 2000s PayPal employees and fo...
4,5,Elliot Page Gets Fan Vote To Play Elon Musk In...,Yahoo,20 hours ago,Tesla founder Elon Musk has a biopic on his li...,https://www.yahoo.com/entertainment/elliot-pag...,Tesla founder Elon Musk has a biopic on his li...
...,...,...,...,...,...,...,...
182,183,"Twitter, now X, to remove blocking feature - Musk",BBC,18/08/2023,The blocking feature will be removed for users...,https://www.bbc.co.uk/news/business-66550959?a...,The blocking feature will be removed for users...
183,184,Zuckerberg 'not holding breath' over Musk cage...,BBC,07/08/2023,Following a post on X in which Mr Musk claimed...,https://www.bbc.co.uk/news/business-66424364,"Meta boss Mark Zuckerberg has said he is ""not ..."
184,185,Musk’s one-year X reign: how shocking has it b...,City A.M.,27/10/2023,"A year after Elon Musk’s Twitter takeover, now...",https://www.cityam.com/musks-one-year-x-reign-...,Featured Submit a story Tell us your story. Fe...
185,186,Regulator sues Musk to force testimony in X probe,BBC,05/10/2023,Financial regulators in the US are suing Elon ...,https://www.bbc.com/news/business-67024538,Financial regulators in the US are suing Elon ...


In [47]:
table.to_csv('output.csv', index=False)

In [48]:
table["TEXT"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 187 entries, 0 to 186
Series name: TEXT
Non-Null Count  Dtype 
--------------  ----- 
166 non-null    object
dtypes: object(1)
memory usage: 1.6+ KB


In [50]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ARTICLE NUMBER  187 non-null    int64 
 1   TITLE           187 non-null    object
 2   AUTHOR          187 non-null    object
 3   DATE            187 non-null    object
 4   DESCRIPTION     187 non-null    object
 5   URL             187 non-null    object
 6   TEXT            166 non-null    object
dtypes: int64(1), object(6)
memory usage: 10.4+ KB


In [51]:
def conv_dates(date_column):
    formatted_dates = []

    for date_value in date_column:
        try:
            # Attempt to parse the input date as a datetime
            input_date = datetime.strptime(date_value, "%d/%m/%Y")
            formatted_dates.append(input_date.strftime("%d/%m/%Y"))
        except ValueError:
            # If parsing fails, assume it's a relative date and proceed with conversion
            today = datetime.today()
            days_ago = int(date_value.split()[0])
            new_date = today - timedelta(days=days_ago)
            formatted_dates.append(new_date.strftime("%d/%m/%Y"))

    return formatted_dates

In [52]:
table = pd.read_csv("output.csv")
table["DATE"]=conv_dates(table["DATE"])

In [54]:
table.to_csv('output.csv', index=False)

In [55]:
table = pd.read_csv("output.csv")

In [56]:
count=0

for i in range(len(table["TEXT"])):
    if pd.isna(table["TEXT"][i]):
        count +=1

count

21

In [57]:
table["TEXT"].fillna(table["DESCRIPTION"], inplace=True)

In [58]:
count=0

for i in range(len(table["TEXT"])):
    if pd.isna(table["TEXT"][i]):
        count +=1

count

0

In [59]:
table['DATE'] = pd.to_datetime(table['DATE'], format='%d/%m/%Y', errors='coerce')

In [60]:
count = (table["DATE"].dt.year < 2022).sum()
count

2

In [61]:
table = table[table["DATE"].dt.year >= 2022]

In [62]:
table

,ARTICLE NUMBER,TITLE,AUTHOR,DATE,DESCRIPTION,URL,TEXT
0,1,Elon Musk started a price war that Tesla can't...,Yahoo News UK,2023-11-11,Tesla has spent the past year slashing its pri...,https://uk.news.yahoo.com/elon-musk-started-pr...,Elon Musk has started an electric-vehicle pric...
1,2,Elon Musk 'bans safety clothes because he hate...,Daily Mail,2023-11-12,Elon Musk discouraged employees from wearing y...,https://www.dailymail.co.uk/news/article-12737...,By James Gordon For Dailymail.com and Reuters...
2,3,Obesity drug Wegovy taken by former UK prime m...,Daily Mail,2023-10-22,New research indicates the obesity drug Wegovy...,https://www.dailymail.co.uk/news/article-12738...,By Noa Halff For Dailymail.Com and Associated ...
3,4,Members of the 'PayPal Mafia' include tech tit...,Yahoo,2023-11-01,"""PayPal mafia"" members Elon Musk, Peter Thiel,...",https://www.yahoo.com/lifestyle/members-paypal...,A group of early 2000s PayPal employees and fo...
4,5,Elliot Page Gets Fan Vote To Play Elon Musk In...,Yahoo,2023-10-24,Tesla founder Elon Musk has a biopic on his li...,https://www.yahoo.com/entertainment/elliot-pag...,Tesla founder Elon Musk has a biopic on his li...
...,...,...,...,...,...,...,...
181,182,Italy could host Musk v Zuckerberg cage fight,BBC,2023-08-12,A planned cage fight between tech leaders Elon...,https://www.bbc.co.uk/news/world-europe-66480636,A planned cage fight between tech leaders Elon...
182,183,"Twitter, now X, to remove blocking feature - Musk",BBC,2023-08-18,The blocking feature will be removed for users...,https://www.bbc.co.uk/news/business-66550959?a...,The blocking feature will be removed for users...
183,184,Zuckerberg 'not holding breath' over Musk cage...,BBC,2023-08-07,Following a post on X in which Mr Musk claimed...,https://www.bbc.co.uk/news/business-66424364,"Meta boss Mark Zuckerberg has said he is ""not ..."
184,185,Musk’s one-year X reign: how shocking has it b...,City A.M.,2023-10-27,"A year after Elon Musk’s Twitter takeover, now...",https://www.cityam.com/musks-one-year-x-reign-...,Featured Submit a story Tell us your story. Fe...


In [63]:
table.to_csv('output.csv', index=False)

In [ ]:
#remove pay walls